## Example using constrained GP model
This is the code used to produce the first example in the paper _'Gaussian processes with linear operator inequality constraints'_, https://arxiv.org/abs/1901.03134

### Imports

In [1]:
### Basic imports ###
import sys, os

# For plotting
import plotly
import plotly.plotly as pltly
import plotly.graph_objs as go
import plotly.offline as pltlyoff
from IPython.display import display, HTML

# This is for plotting as static images (to show on e.g. GitHub)
import plotly.io as pio
from IPython.display import Image

# Numerics
import numpy as np
import scipy as sp
import itertools
import pyDOE

### Custom files ###

# Path to custom plotly module 'GPPlotly' for plotting 
# can be downloaded at https://github.com/cagrell/gp_plotly
dir_GPPlotly = 'C:\\Data\\git repos\\gp_plotly\\'
sys.path.append(dir_GPPlotly) 

# Path to the constrained GP moule 
# can be downloaded at https://github.com/cagrell/gp_constr
dir_gp_constr = 'C:\\Data\\git repos\\gp_constr\\'
sys.path.append(dir_gp_constr) 

# Import
from GPPlotly.plottingfunctions import PlotGP2d, add_traces_to_fig
from GPConstr.model import kernel_RBF, GPmodel, Constraint

### Setup notebook ###
pltlyoff.init_notebook_mode(connected=True)
print('Python version', sys.version)

Loading constrained GP module from C:\Data\git repos\gp_constr
Loading R wrapper...
Running R from rpy2: R version 3.4.3 (2017-11-30)


Python version 3.6.3 (v3.6.3:2c5fed8, Oct  3 2017, 18:11:49) [MSC v.1900 64 bit (AMD64)]


## 1. Define function for generating synthetic test/training data


In [2]:
# Function to emulate/estimate
def fun(x):
    return (np.arctan(20*x - 10) - np.arctan(-10))/3

## 2. Regression with noise

### 2.1. Generate synthetic training data

In [3]:
# Design data - with noise
n = 50
noise_std = 0.2
x_design = np.random.uniform(0.1, 0.8, n)
y_design = fun(x_design) + np.random.normal(0, noise_std, n)

# For plotting
x_test = np.linspace(0, 1, 500)
y_true = fun(x_test)

### 2.2. Define GP model

In [4]:
# Set up model
ker = kernel_RBF(variance = 0.5, lengthscale = [0.1])
model = GPmodel(kernel = ker, likelihood = 1, mean = 0) 

# Add the training data
model.X_training = x_design.reshape(-1, 1)
model.Y_training = y_design

### 2.2.3. Include constraints

In [5]:
# Helper functions for constraints

def constant_function(val):
    """ Return the constant function"""
    def fun(x):
        return np.array([val]*x.shape[0])
    
    return fun

def fun_UB(x):
    """ Upper bound function """
    return np.log(30*x.flatten() + 1)/3 + 0.1


In [6]:
# Define constraints for bounding the function and its derivative
constr_bounded = Constraint(LB = constant_function(0), UB = fun_UB)
constr_deriv = Constraint(LB = constant_function(0), UB = constant_function(float('Inf')))

In [7]:
# Add constraints to model
model.constr_bounded = constr_bounded
model.constr_deriv = [constr_deriv] # Add list of constraints for multi-dimensional functions

In [8]:
# Set virtual points manually 
model.reset()
model.constr_bounded.Xv = pyDOE.lhs(n = 1, samples = 20)
model.constr_deriv[0].Xv = pyDOE.lhs(n = 1, samples = 6)

print(model)

----- GP model ----- 
 mean = 0 
 likelihood = 1 
 kernel: 
   type = RBF 
   input dim = 1 
   lenghtscale = [0.1] 
   variance = 0.5 
 constraint: 
   f [20], df/dx_1 [6] 
   constr_likelihood = 1e-06 
---------------------


In [ ]:
# Search for a suitable set of virtual observation locations where the constraint is imposed
#df = model.find_XV_subop(bounds = [(0.001, 1)], p_target = 0.9, max_iterations = 100, min_prob_unconstr_xv = -1, i_range = [0, 1], opt_method = 'shgo')

In [ ]:
# Search for a suitable set of virtual observation locations where the constraint is imposed
#df = model.find_XV_subop(bounds = [(0.001, 1)], p_target = 0.9, max_iterations = 100, min_prob_unconstr_xv = -1, i_range = [0, 1], opt_method = 'differential_evolution')

In [ ]:
# Optimize unconstrained
model.optimize(include_constraint = False, fix_likelihood = False)
print(model)

In [ ]:
# Plot unconstrained GP
mean_unconstr, cov_unconstr = model.calc_posterior_unconstrained(x_test.reshape(-1, 1), full_cov = True)
mean_unconstr = np.array(mean_unconstr).flatten()
var_unconstr = np.diagonal(cov_unconstr)

num_samples = 30
show_samplepaths = True
samplepaths_unconstr = []
if show_samplepaths: samplepaths_unconstr = np.random.multivariate_normal(mean_unconstr, cov_unconstr, num_samples).T

fig_unconstr_1 = PlotGP2d(x_mean = x_test, mean = mean_unconstr, var = var_unconstr,
                        x_obs = model.X_training[:,0], y_obs = model.Y_training, 
                        num_std = 1.2815,
                        x_true = x_test, y_true = y_true,
                        samplepaths = samplepaths_unconstr,
                        title = 'Unconstrained GP', xrange = [0, 1], yrange = [-1.7, 1.7], smoothing = True)

trace_UB = go.Scatter(x = x_test, y = model.constr_bounded.UB(x_test), mode = 'lines', name = 'Upper bound', line = dict(color = ('rgb(0, 0, 0)'), shape = 'spline', width = 1))
trace_LB = go.Scatter(x = x_test, y = model.constr_bounded.LB(x_test), mode = 'lines', name = 'Lower bound', line = dict(color = ('rgb(0, 0, 0)'), shape = 'spline', width = 1))

fig_unconstr_1 = add_traces_to_fig(fig_unconstr_1, [trace_UB, trace_LB])

pltlyoff.iplot(fig_unconstr_1, filename='')

In [ ]:
# Plot model with both constraints

mean, var, perc, mode, samples = model.calc_posterior_constrained(x_test.reshape(-1, 1), compute_mode = False, num_samples = 10000, save_samples = 30, algorithm = 'minimax_tilting', resample = False)

mean = np.array(mean).flatten()
p_lower = perc[0]
median = perc[1]
p_upper = perc[2]
p_label = '[p{}, p{}]'.format(10, 90)

samplepaths_Z = np.array(samples)

fig_both = PlotGP2d(x_mean = x_test, mean = mean,
                        x_obs = model.X_training[:,0], y_obs = model.Y_training, 
                        p_lower = p_lower, p_upper = p_upper, p_label = p_label,
                        samplepaths =  samplepaths_Z,
                        x_true = x_test, y_true = y_true,
                        title = 'Both constraints', xrange = [0, 1], yrange = [-1.7, 1.7], smoothing = True)

trace_UB = go.Scatter(x = x_test, y = model.constr_bounded.UB(x_test), mode = 'lines', name = 'Upper bound', line = dict(color = ('rgb(0, 0, 0)'), shape = 'spline', width = 1))
trace_LB = go.Scatter(x = x_test, y = model.constr_bounded.LB(x_test), mode = 'lines', name = 'Lower bound', line = dict(color = ('rgb(0, 0, 0)'), shape = 'spline', width = 1))
trace_XV_bounded = go.Scatter(x = model.constr_bounded.Xv.flatten(), y = np.zeros(model.constr_bounded.Xv.shape[0]), mode = 'markers', name = 'Xv - boundedness', marker = dict(symbol = 'line-ns-open', color = ('rgb(0, 0, 0)')))
trace_XV_mon = go.Scatter(x = model.constr_deriv[0].Xv.flatten(), y = np.zeros(model.constr_deriv[0].Xv.shape[0]), mode = 'markers', name = 'Xv - monotonicity', marker = dict(symbol = 'x-thin-open', color = ('rgb(0, 0, 0)')))

fig_both = add_traces_to_fig(fig_both, [trace_UB, trace_LB, trace_XV_bounded, trace_XV_mon])
#fig_both = add_traces_to_fig(fig_both, [trace_UB, trace_LB, trace_XV_bounded])

pltlyoff.iplot(fig_both, filename='')


## Plot likelihood

In [ ]:
# Some helper functions
def Create_meshdata(plotfun, x_range, y_range, *args, **kwargs):
    """
    Function for creating meshgrid data for surface plotting.
    Inputs:
        plotfun - function of (x, y, *args, **kwargs)
        x_range, y_range
    Outputs:
        X, Y - meshgrid of x_range and y_range
        Z - plotfun(X, Y, *args, **kwargs)
    """
    
    # Create meshgrid
    X, Y = np.meshgrid(x_range, y_range)
    
    # Evaluate function
    vfun = np.vectorize(plotfun)
    Z = [vfun(x, y, *args, **kwargs) for x, y in zip(X, Y)]
    
    return X, Y, Z

def loglik_unconstr(x, y):
    """
    x = kernel lengthscale
    y = kernel variance
    """
    model.kernel.variance = y
    model.kernel.lengthscale = [x]
    model.reset()
    return model._loglik_unconstrained()

def loglik_constr(x, y):
    """
    x = kernel lengthscale
    y = kernel variance
    """
    model.kernel.variance = y
    model.kernel.lengthscale = [x]
    model.reset()
    
    v_loglik_unconstr = model._loglik_unconstrained() # P(Y)
    v_loglik_constr = np.log(model.constrprob_Xv(posterior = True, algorithm = 'minimax_tilting', n = 100)) # P(C|Y)
            
    return v_loglik_unconstr + v_loglik_constr # P(Y, C)

def loglik_constr_cond(x, y):
    """
    x = kernel lengthscale
    y = kernel variance
    """
    model.kernel.variance = y
    model.kernel.lengthscale = [x]
    model.reset()
    
    v_loglik_unconstr = model._loglik_unconstrained() # P(Y)
    v_loglik_constr_cond = np.log(model.constrprob_Xv(posterior = False, algorithm = 'minimax_tilting', n = 100)) # P(C)
    v_loglik_constr = np.log(model.constrprob_Xv(posterior = True, algorithm = 'minimax_tilting', n = 100)) # P(C|Y)
                                 
    return (v_loglik_constr + v_loglik_constr_cond - v_loglik_unconstr) # P(Y|C)

def Q(x, y, g):
    """
    x = kernel lengthscale
    y = kernel variance
    """
    model.kernel.variance = y
    model.kernel.lengthscale = [x]
    model.reset()
    
    #return model._EM_Q(g)
    return model._EM_Q_check(n = 1000) 
    
def compute_Q_meshdata(lengthscale_range, variance_range):
    optimal_l = model.kernel.lengthscale[0]
    optimal_var = model.kernel.variance
    
    g = model._EM_g()
    
    # Create meshgrid
    X, Y = np.meshgrid(lengthscale_range, variance_range)
    
    # Evaluate function
    Z = np.zeros(X.shape)
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            Z[i][j] = Q(X[i][j], Y[i][j], g)
    
    
    # Reset model values
    model.kernel.variance = optimal_var
    model.kernel.lengthscale = [optimal_l]
    model.reset()
    
    return X, Y, Z
    
def compute_loglik_meshdata(lengthscale_range, variance_range, constr = False, conditional = False):
    optimal_l = model.kernel.lengthscale[0]
    optimal_var = model.kernel.variance
    
    if constr:
        if conditional:
            X1, X2, Y = Create_meshdata(loglik_constr_cond, lengthscale_range, variance_range)                    
        else:
            X1, X2, Y = Create_meshdata(loglik_constr, lengthscale_range, variance_range)
    else:
        X1, X2, Y = Create_meshdata(loglik_unconstr, lengthscale_range, variance_range)
    
    # Reset model values
    model.kernel.variance = optimal_var
    model.kernel.lengthscale = [optimal_l]
    model.reset()
    
    return X1, X2, Y

In [ ]:
# Optimize unconstrained
model.likelihood = 1
model.reset()
model.optimize(include_constraint = False, fix_likelihood = True)
print(model)

In [ ]:
# Compute unconstrained likelihood
lengthscale_range = np.linspace(0.05, 1, 20)
variance_range = np.linspace(0.1, 3, 20)

X1, X2, Y = compute_loglik_meshdata(lengthscale_range, variance_range, constr = False)

contour_unconstr = go.Contour(
                                x = lengthscale_range, y = variance_range, z = Y, name = 'Unconstrained log-likelihood',
                                autocontour = False, contours=dict(start = -52, end = -50, size = 0.2),
                             )
max_unconstr = go.Scatter(x = [model.kernel.lengthscale[0]], y = [model.kernel.variance], mode = 'markers', name = 'current')

data = [contour_unconstr, max_unconstr]
layout = go.Layout(title = 'Unconstrained log-likelihood', xaxis=dict(title = 'lengthscale'), yaxis=dict(title = 'variance'))
fig = go.Figure(data = data, layout = layout)
pltlyoff.iplot(fig, filename='')
print('Optimal: {}, {}'.format(model.kernel.lengthscale[0], model.kernel.variance))

In [ ]:
# Optimize constrained - global

#opt_args = {'maxiter': 10}
opt_args = {}

bound_lik = None
bound_ker_var = (0.1, 3)
bound_ker_len = [(0.1, 2)]*1

bounds = [bound_lik] if bound_lik is not None else []
bounds = bounds + [bound_ker_var] + bound_ker_len
#bounds = None
model._optimize_constrained(fix_likelihood = True, conditional = False, opt_method = 'differential_evolution', 
                            algorithm = 'minimax_tilting', n = 10, opt_args = opt_args, bounds = bounds)
print(model)

In [ ]:
# Optimize constrained - local

#opt_args = {'maxiter': 10}
opt_args = {}

bounds = [(1e-6, None)]*2

model._optimize_constrained(fix_likelihood = True, conditional = False, opt_method = 'L-BFGS-B', 
                            algorithm = 'minimax_tilting', n = 10, opt_args = opt_args, bounds = bounds)
print(model)

In [ ]:
#model.likelihood = 0.1
model.reset()
model.constrprob_Xv(n = 10)

In [ ]:
# Optimize constrained
model.constr_likelihood = 1E-6
model.kernel.lengthscale = [0.1]
model.kernel.variance = 0.5
model.reset()

opt_args = {'options' : {'maxtime ':10}}
opt_args = {}
model._optimize_constrained(fix_likelihood = True, opt_method = 'shgo', algorithm = 'minimax_tilting', n = 10, opt_args = opt_args)
print(model)

In [ ]:
# Compute constrained likelihood
import time
#model.constr_likelihood = 1E-3
model.reset()
lengthscale_range = np.linspace(0.05, 1, 20)
variance_range = np.linspace(0.1, 3, 20)

t0 = time.time()
X1, X2, Y = compute_loglik_meshdata(lengthscale_range, variance_range, constr = True)
print('time ', time.time() - t0)

In [ ]:
contour_constr = go.Contour(
                                x = lengthscale_range, y = variance_range, z = Y, name = 'Constrained log-likelihood',
                                autocontour = True, contours=dict(start = -54, end = -50, size = 0.15),
                             )
max_constr = go.Scatter(x = [model.kernel.lengthscale[0]], y = [model.kernel.variance], mode = 'markers', name = 'current')

data = [contour_constr, max_unconstr, max_constr]
layout = go.Layout(title = 'Constrained log-likelihood, noise_var = ' + str(model.constr_likelihood), xaxis=dict(title = 'lengthscale'), yaxis=dict(title = 'variance'))
fig = go.Figure(data = data, layout = layout)
pltlyoff.iplot(fig, filename='')
print('Optimal: {}, {}'.format(model.kernel.lengthscale[0], model.kernel.variance))

In [ ]:
# Compute Q
X1, X2, Y = compute_Q_meshdata(lengthscale_range, variance_range)

In [ ]:
contour_Q = go.Contour(
                                x = lengthscale_range, y = variance_range, z = Y, name = 'Q',
                                autocontour = True, contours=dict(start = -54, end = -50, size = 0.15),
                             )

data = [contour_Q]
layout = go.Layout(title = 'Q, noise_var = ' + str(model.constr_likelihood), xaxis=dict(title = 'lengthscale'), yaxis=dict(title = 'variance'))
fig = go.Figure(data = data, layout = layout)
pltlyoff.iplot(fig, filename='')

In [ ]:
# Compute Q
X1, X2, Y = compute_Q_meshdata(lengthscale_range, variance_range)

In [ ]:
contour_Q = go.Contour(
                                x = lengthscale_range, y = variance_range, z = Y, name = 'Q',
                                autocontour = True, contours=dict(start = -54, end = -50, size = 0.15),
                             )

data = [contour_Q]
layout = go.Layout(title = 'Q, noise_var = ' + str(model.constr_likelihood), xaxis=dict(title = 'lengthscale'), yaxis=dict(title = 'variance'))
fig = go.Figure(data = data, layout = layout)
pltlyoff.iplot(fig, filename='')

In [ ]:
contour_constr = go.Contour(
                                x = lengthscale_range, y = variance_range, z = Y, name = 'Constrained log-likelihood',
                                autocontour = False, contours=dict(start = -54, end = -50, size = 0.15),
                             )
max_constr = go.Scatter(x = [model.kernel.lengthscale[0]], y = [model.kernel.variance], mode = 'markers', name = 'current')

data = [contour_unconstr, max_unconstr, max_constr]
layout = go.Layout(title = 'Constrained log-likelihood, noise_var = ' + str(model.constr_likelihood), xaxis=dict(title = 'lengthscale'), yaxis=dict(title = 'variance'))
fig = go.Figure(data = data, layout = layout)
pltlyoff.iplot(fig, filename='')
print('Optimal: {}, {}'.format(model.kernel.lengthscale[0], model.kernel.variance))

In [ ]:
# Compute constrained likelihood
import time
#model.constr_likelihood = 1E-3
model.reset()
lengthscale_range = np.linspace(0.05, 5, 30)
variance_range = np.linspace(0.1, 1, 30)

t0 = time.time()
X1, X2, Y = compute_loglik_meshdata(lengthscale_range, variance_range, constr = True, conditional = True)
print('time ', time.time() - t0)

In [ ]:
contour_constr_cond = go.Contour(
                                x = lengthscale_range, y = variance_range, z = Y, name = 'Constrained log-likelihood',
                                autocontour = False, contours=dict(start = 45, end = 48, size = 0.5),
                             )
max_constr_cond = go.Scatter(x = [model.kernel.lengthscale[0]], y = [model.kernel.variance], mode = 'markers', name = 'current')

data = [contour_constr_cond, max_unconstr, max_constr, max_constr_cond]
layout = go.Layout(title = 'Constrained conditional log-likelihood, noise_var = ' + str(model.constr_likelihood), xaxis=dict(title = 'lengthscale'), yaxis=dict(title = 'variance'))
fig = go.Figure(data = data, layout = layout)
pltlyoff.iplot(fig, filename='')
print('Optimal: {}, {}'.format(model.kernel.lengthscale[0], model.kernel.variance))

In [ ]:
contour_constr_cond = go.Contour(
                                x = lengthscale_range, y = variance_range, z = Y, name = 'Constrained log-likelihood',
                                autocontour = False, contours=dict(start = 45, end = 51, size = 0.5),
                             )
max_constr_cond = go.Scatter(x = [model.kernel.lengthscale[0]], y = [model.kernel.variance], mode = 'markers', name = 'current')

data = [contour_constr_cond, max_unconstr, max_constr, max_constr_cond]
layout = go.Layout(title = 'Constrained conditional log-likelihood, noise_var = ' + str(model.constr_likelihood), xaxis=dict(title = 'lengthscale'), yaxis=dict(title = 'variance'))
fig = go.Figure(data = data, layout = layout)
pltlyoff.iplot(fig, filename='')
print('Optimal: {}, {}'.format(model.kernel.lengthscale[0], model.kernel.variance))

## Test new alg bootstrap

In [9]:
def print_constr_loglik(model, intermediate = True):
    v_loglik_unconstr = model._loglik_unconstrained() # P(Y)
    v_loglik_constr = np.log(model.constrprob_Xv(posterior = True, algorithm = 'minimax_tilting', n = 1000)) # P(C|Y)
    
    if intermediate:
        print('P(Y)', v_loglik_unconstr)
        print('P(C|Y)', v_loglik_constr)
    
    print('P(Y, C)', v_loglik_unconstr + v_loglik_constr)

In [62]:
model.likelihood = 1
model.kernel.variance = 0.5
model.kernel.lengthscale = [0.1]
model.reset()

In [63]:
# Optimize unconstrained
model.optimize(include_constraint = False, fix_likelihood = False)
print(model)
print_constr_loglik(model)

..Running optimization for unconstrained GP ... DONE - Total time: 0.088 seconds
----- GP model ----- 
 mean = 0 
 likelihood = 0.03548499979269271 
 kernel: 
   type = RBF 
   input dim = 1 
   lenghtscale = [0.29251104] 
   variance = 0.34576448836813417 
 constraint: 
   f [20], df/dx_1 [6] 
   constr_likelihood = 1e-06 
---------------------
P(Y) 5.041485472059151
P(C|Y) -8.139435123914025
P(Y, C) -3.097949651854874


In [65]:
# EM updates
#bounds = [(1e-6, 1), (1e-6, 30), (1e-6, 10)]
bounds = None

#model._EM_update(fix_likelihood = False, bounds = bounds, n = 1000, verbatim = True, opt_method = 'shgo')
for i in range(10):
    model._EM_update(fix_likelihood = False, bounds = bounds, n = 1000, verbatim = True)
print(model)
print_constr_loglik(model)

..Running calculation of g(theta~) ... DONE - time: 0.044 seconds
..Running optimization (L-BFGS-B) ... DONE - time: 0.335 seconds
..Running calculation of g(theta~) ... DONE - time: 0.047 seconds
..Running optimization (L-BFGS-B) ... DONE - time: 0.095 seconds
..Running calculation of g(theta~) ... DONE - time: 0.047 seconds
..Running optimization (L-BFGS-B) ... DONE - time: 0.139 seconds
..Running calculation of g(theta~) ... DONE - time: 0.035 seconds
..Running optimization (L-BFGS-B) ... DONE - time: 0.094 seconds
..Running calculation of g(theta~) ... DONE - time: 0.047 seconds
..Running optimization (L-BFGS-B) ... DONE - time: 0.120 seconds
..Running calculation of g(theta~) ... DONE - time: 0.040 seconds
..Running optimization (L-BFGS-B) ... DONE - time: 0.492 seconds
..Running calculation of g(theta~) ... DONE - time: 0.047 seconds
..Running optimization (L-BFGS-B) ... DONE - time: 0.430 seconds
..Running calculation of g(theta~) ... DONE - time: 0.031 seconds
..Running optimiz

In [ ]:
# Optimize constrained local
opt_args = {}

bounds = [(1e-6, None)]*3

model._optimize_constrained(fix_likelihood = False, conditional = False, opt_method = 'L-BFGS-B', 
                            algorithm = 'minimax_tilting', n = 10, opt_args = opt_args, bounds = bounds)
print(model)
print_constr_loglik(model)

In [ ]:
# Optimize constrained global
opt_args = {}

#bounds = [(1e-6, None)]*2
bounds = [(1e-1, 1), (1e-6, 30), (1e-6, 10)]

model._optimize_constrained(fix_likelihood = False, conditional = False, opt_method = 'differential_evolution', 
                            algorithm = 'minimax_tilting', n = 10, opt_args = opt_args, bounds = bounds)
print(model)
print_constr_loglik(model)

In [27]:
model.kernel.lengthscale = [0.1]
model.kernel.variance = 0.5
model.likelihood = 1
print_constr_loglik(model)

P(Y) -52.149008618996795
P(C|Y) -11.03796754738172
P(Y, C) -63.186976166378514


In [ ]:
model.optimize(include_constraint = False, fix_likelihood = True)
print(model)
print_constr_loglik(model)

In [ ]:
#bounds = [(0.1, None)]*2

In [ ]:
model._EM_update(fix_likelihood = False, bounds = None, n = 1000, verbatim = True)
print(model)
print_constr_loglik(model)

In [ ]:
# Optimize constrained - local

#opt_args = {'maxiter': 10}
opt_args = {}

bounds = [(1e-6, None)]*2

model._optimize_constrained(fix_likelihood = True, conditional = False, opt_method = 'L-BFGS-B', 
                            algorithm = 'minimax_tilting', n = 10, opt_args = opt_args, bounds = bounds)
print(model)
print_constr_loglik(model)

In [ ]:
g = model._EM_g()
Q, zGz = model._EM_Q(g)

print(Q)
print(zGz)

In [ ]:
Q_check, zGz_check = model._EM_Q_check(n = 1000)
print(Q_check)
print(zGz_check)

In [ ]:
C_sim, Gamma, Gamma_inv, L = model._EM_Q_check(n = 100)

In [ ]:
tmp = np.zeros(C_sim.shape[0])
tmp_2 = None

for i in range(C_sim.shape[0]):
    z = np.matrix(list(model.Y_centered.flatten()) + list(C_sim[i])).T
    tmp[i] = (z.T*Gamma_inv*z)[0,0]
    if tmp_2 is None:
        tmp_2 = z*z.T
    else:
        tmp_2 = tmp_2 + z*z.T

In [ ]:
avg_1 = np.array(tmp).mean()
print(avg_1)

In [ ]:
E_z_zt = tmp_2/C_sim.shape[0]
avg_2 = np.sum(np.diag(E_z_zt*Gamma_inv))

avg_2

In [ ]:
print(model)

In [ ]:
print(model)

In [ ]:
# Plot 1D Q

def Q_calc(x, g = None):
    l = model.kernel.lengthscale[0]
    model.kernel.lengthscale[0] = x
    model.reset()
    
    if g is None:
        gg = model._EM_g()
    else:
        gg = g
    
    Q = model._EM_Q(gg)
    
    model.kernel.lengthscale[0] = l
    model.reset()
    
    return Q

def Q_check_calc(x):
    l = model.kernel.lengthscale[0]
    model.kernel.lengthscale[0] = x
    model.reset()
    
    Q, zGz, H, det_1, det_2 = model._EM_Q_check(n = 1000)
    
    model.kernel.lengthscale[0] = l
    model.reset()
    
    return H

def loglik_all(x):
    l = model.kernel.lengthscale[0]
    model.kernel.lengthscale[0] = x
    model.reset()
    
    v_loglik_unconstr = model._loglik_unconstrained() # P(Y)
    v_loglik_constr = np.log(model.constrprob_Xv(posterior = True, algorithm = 'minimax_tilting', n = 1000)) # P(C|Y)
            
    model.kernel.lengthscale[0] = l
    model.reset()
    
    return v_loglik_unconstr, v_loglik_constr, v_loglik_unconstr + v_loglik_constr


In [ ]:
# Optimize unconstrained
model.optimize(include_constraint = False, fix_likelihood = True)
print(model)

In [ ]:
# Q 
x = np.linspace(0.1, 0.23, 20)
g_opt = model._EM_g()
Q_opt = np.array([Q_calc(xx, g_opt) for xx in x])
Q_current = np.array([Q_calc(xx) for xx in x])

In [ ]:
# log P(Y, C)
log_P_Y = np.zeros(x.shape[0])
log_P_CgY = np.zeros(x.shape[0])
log_P_YC = np.zeros(x.shape[0])

for i in range(x.shape[0]):
    log_P_Y[i], log_P_CgY[i], log_P_YC[i] = loglik_all(x[i])


In [ ]:
x

In [ ]:
# Q check
Q_check = np.array([Q_check_calc(xx) for xx in x])

In [ ]:
# Plot
trace_loglik = go.Scatter(x = x, y = log_P_YC, mode = 'lines', name = 'log P(Y, C)')
trace_loglik_2 = go.Scatter(x = x, y = log_P_Y, mode = 'lines', name = 'log P(Y)')

trace_Q_opt = go.Scatter(x = x, y = Q_opt, mode = 'lines', name = 'Q_opt')
trace_Q_current = go.Scatter(x = x, y = Q_current, mode = 'lines', name = 'Q_current')
trace_Q_check = go.Scatter(x = x, y = Q_check, mode = 'lines', name = 'ELBO')

data = [trace_loglik, trace_Q_opt, trace_Q_current, trace_Q_check, trace_loglik_2]
layout = go.Layout(title = '', xaxis=dict(title = 'lengthscale'), yaxis=dict(title = 'log likelihood'))
fig = go.Figure(data = data, layout = layout)
pltlyoff.iplot(fig, filename='')

In [ ]:
model.kernel.lengthscale = [0.18]
model.reset()
print(model)
print_constr_loglik(model)

In [ ]:
g_opt = model._EM_g()
Q_opt = model._EM_Q(g_opt)
print(Q_opt)

In [ ]:
Q_check, zGz, H, det_1, det_2 = model._EM_Q_check(n = 1000)

print(Q_check)
print(zGz)
print(H)
print(det_1)
#print(det_2)
print(Q_check + H)

In [ ]:
(-54.26315345870914)-Q_check

In [ ]:
n = model._num_virtuial_pts() + model.Y_training.shape[0]
#-det_1 - (n/2)*np.log(2*np.pi) -0.5*zGz
- (n/2)*np.log(2*np.pi) -0.5*zGz

In [ ]:
L2T_K_x_xv = model._calc_L2T(model.X_training)
L1L2T_K_xv_xv = model._calc_L1L2()
Gamma = np.block([[model.K_w, L2T_K_x_xv], [L2T_K_x_xv.T, L1L2T_K_xv_xv + model.constr_likelihood*np.identity(n = L1L2T_K_xv_xv.shape[0])]])
L = np.linalg.cholesky(Gamma)
np.log(np.diag(L)).sum()

In [ ]:
Gamma.shape

In [ ]:
#np.diag(model.K_w)
L2T_K_x_xv = model._calc_L2T(model.X_training)
L2T_K_x_xv.shape
L1L2T_K_xv_xv = model._calc_L1L2()
L1L2T_K_xv_xv.shape

np.identity(n = L1L2T_K_xv_xv.shape[0]).shape

In [ ]:
L = np.linalg.cholesky(model.B1)
np.diag(L)

In [ ]:
print(Q_check)
print(model._loglik_unconstrained())

In [ ]:
#model.B1
B1_inv = np.linalg.inv(model.B1)



In [ ]:
print(np.log(np.linalg.det(model.B1)))

In [ ]:
L = np.linalg.cholesky(model.B1)

In [ ]:
#np.log(np.diag(L)).sum()

2*np.log(np.diag(L)).sum() 

In [ ]:
from GPConstr.r_functions.python_wrappers import mtmvnorm, moments_from_samples, rtmvnorm

In [ ]:
y_obs = 2
rho = 0.5
a = 0.5
b = 1

# Distribution of c|Y
cond_mean = rho*y_obs
cond_var = 1 - rho**2

# P(Y)
ln_P_Y = -(1/2)*np.log(2*np.pi) - y_obs**2
ln_P_CgY = np.log(sp.stats.norm.cdf(b, loc=cond_mean, scale=cond_var) - sp.stats.norm.cdf(a, loc=cond_mean, scale=cond_var))

print('logP(Y)', ln_P_Y)
print('logP(C|Y)', ln_P_CgY)
print('logP(Y, C)', ln_P_Y + ln_P_CgY)

# EM
Gamma = np.matrix([[1, rho], [rho, 1]])
Gamma_inv = np.linalg.inv(Gamma)

# Moments of truncated variable
cm, cs = mtmvnorm(np.array([0]), np.matrix([[1]]), [a], [b])
#cm, cs = moments_from_samples(1000, np.array([0]), np.matrix([[1]]), [a], [b])
cm = cm[0]
cs = cs[0][0]

# g matrix
g = np.matrix([[y_obs**2, y_obs*cm], [y_obs*cm, cs + cm**2]])


# Q
# Sample from constraint distribution
C_sim = rtmvnorm(100, np.array([0]), np.matrix([[1]]), [a], [b]).flatten()

Q = -0.5*np.diag(g*Gamma_inv).sum() - 0.5*np.log(np.linalg.det(Gamma)) -(2/2)*np.log(2*np.pi)
#H = (1/2)*np.log(2*np.pi) + np.log(sp.stats.norm.cdf(b) - sp.stats.norm.cdf(a)) + (C_sim**2).mean() # Wrong ?

Z = np.exp(ln_P_CgY)
alpha = (a - cond_mean)/np.sqrt(cond_var)
beta = (b - cond_mean)/np.sqrt(cond_var)

H = np.log(np.sqrt(2*np.pi*np.exp(1))*np.sqrt(cond_var)*Z) + (alpha*sp.stats.norm.pdf(alpha) - beta*sp.stats.norm.pdf(beta))/(2*Z)

print('')
print('Q', Q)
print('H', H)
print('ELBO', Q + H)

In [ ]:
# Sample from constraint distribution
C_sim = rtmvnorm(100, np.array([0]), np.matrix([[1]]), [a], [b]).flatten()

(C_sim**2).mean()

In [ ]:
Z = np.exp(ln_P_CgY)
alpha = (a - cond_mean)/np.sqrt(cond_var)
beta = (b - cond_mean)/np.sqrt(cond_var)

np.log(np.sqrt(2*np.pi*np.exp(1))*np.sqrt(cond_var)*Z) + (alpha*sp.stats.norm.pdf(alpha) - beta*sp.stats.norm.pdf(beta))/(2*Z)

In [ ]:
# Optimize unconstrained
model.optimize(include_constraint = False, fix_likelihood = True)
print(model)

In [ ]:
sp.stats.norm.cdf(1/2.2, loc=0, scale=1)